In [1]:
import torch
import math
import matplotlib.pyplot as plt

torch.set_grad_enabled(False)

In [63]:
class Module(object) :
    def __init__(self):
        super().__init__()
    
    def forward(self , *input):
        raise  NotImplementedError
        
    def backward(self , *gradwrtoutput):
        raise  NotImplementedError
        
    def param(self): # These are the layers of the network
        return  []

In [64]:
class Sequential(Module) :
    def __init__(self, param):
        super().__init__()
        self.model = (param)
        self.loss = LossMSE()
    
    def forward(self, x):
        for layer in self.model:
            x = layer.forward(x)
        return x
    
    def backward(self, output, target):
        # Loss = self.loss.forward(x, y)
        
        grad = self.loss.backward(output, target)
        for layer in reversed(self.model):
            grad = layer.backward(grad)
        return

In [65]:
class Linear(Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        
        # Initialization of the weights and the bias
        param = 1 / math.sqrt(in_features)
        self.weight = torch.empty(out_features, in_features).uniform_(-param, param)
        self.bias = torch.empty(out_features).uniform_(-param, param)
        
    def forward(self, x):
        return torch.mm(x, self.weight.t()) + self.bias
    
    def set_Lr(self, lr):
        self.lr=lr
        return
        
    def backward(self, x, grad):
        lr = 0.001
        self.weight = self.weight - lr * self.x.t().mm(grad) 
        self.bias = self.bias - lr * grad * 1
        grad = grad.mm(self.weight.t())
        return grad
    
    def weight(self):
        return self.weight
    
    def bias(self):
        return self.bias

In [66]:
class LossMSE(object):
    def __init__(self):
        super().__init__() 
    
    def forward(self, data_input, data_target):
        loss = (data_input - data_target).pow(2).sum()
        return loss
    
    def backward(self, data_input, data_target):
        dloss = 2*(data_input - data_target)
        return dloss

In [67]:
class ReLU(Module) :
    def __init__(self):
        super().__init__()
    
    def forward(self, x):
        for k in range(x.size(0)):
            for i in range(2):
                x[k][i] = max(0, x[k][i].tolist())
        return x
    
    def backward(self, x):
        if x > 0:
            return 1
        if x <= 0:
            return 0

In [68]:
class Tanh(object) :
    def __init__(self, ):
        super().__init__() 
    
    def  forward(self, x):
        return torch.div(x.exp() - (-x).exp(), x.exp() + (-x).exp())
        
    def  backward(self, *gradwrtoutput):
        return 1 - torch.div(x.exp() - (-x).exp(), x.exp() + (-x).exp())**2
        
    def  param(self):
        return

In [69]:
def create_random_batch(input_size, mini_batch_size):
    
    # This function return a 2D tensor that is the rando selection of inputs for our
    # stochastic gradient method, taking in count the number of mini_batches.
    
    # We suppose here that our mini_batch_size is well chosen taking in count the fact
    # that it divides input_size.
    
    # Initialization
    L = int(input_size / mini_batch_size)
    new_batch = torch.ones(L, mini_batch_size)
    
    indices = torch.randperm(input_size)
    for k in range(L):
        new_batch[k] = indices[k * mini_batch_size : (k+1) * mini_batch_size]
    
    return new_batch

In [70]:
def train_model(model, train_input, train_classes, nb_epochs, mini_batch_size):
    
    h_step = 1e-3
    
    for epoch in range(nb_epochs):
        random_batches = create_random_batch(train_input.size(0), mini_batch_size).tolist()
        for batch in range(mini_batch_size):
            output = model.forward(train_input[random_batches[batch]])
            #loss = LossMSE(output, train_classes[random_batches[batch]])
            model.backward(output, train_classes[random_batches[batch]])

In [71]:
def compute_nb_errors(model, data_input, data_target, mini_batch_size):

    nb_data_errors = 0

    for b in range(0, data_input.size(0), mini_batch_size):
        _, _, result = model.forward(data_input.narrow(0, b, mini_batch_size))
        predicted_classes = torch.floor(result)
        for k in range(mini_batch_size):
            if data_target[b + k] != predicted_classes[k]:
                nb_data_errors = nb_data_errors + 1

    return nb_data_errors

In [72]:
def create_problem(nb_samples):
    
    # Remark: the function .uniform return a uniform distribution on [0,1) instead of [0,1],
    # but in our case it's not a problem since it is only a train and a test set on a circle
    # that do not touch the border of the set [0,1]^2.
    train_input = torch.empty(nb_samples, 2).uniform_(0, 1)
    test_input = torch.empty(nb_samples, 2).uniform_(0, 1)
    
    # Radius of our circle
    R = 1 / math.sqrt(2 * math.pi)
    
    train_classes = train_input.sub(0.5).pow(2).sum(1).sub(R**2).sign().sub(1).div(-2).long()
    test_classes = test_input.sub(0.5).pow(2).sum(1).sub(R**2).sign().sub(1).div(-2).long()
    
    return train_input, train_classes, test_input, test_classes

In [73]:
def get_tests(n):
    M = []
    for k in range (0, n):
        L = []
        _, _, test_input, test_classes =  create_problem(1000)
        L.append(test_input)
        L.append(test_classes)
        M.append(L)
    return M

In [74]:
train_input, train_classes, _, _ = create_problem(1000)
nb_epochs = 10
mini_batch_size = 200

model = Sequential([Linear(2,200), ReLU(), Linear(200,1), ReLU()])
train_model(model, train_input, train_classes, nb_epochs, mini_batch_size)

<ipython-input-70-6688f699d5a4>:8: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  output = model.forward(train_input[random_batches[batch]])
<ipython-input-70-6688f699d5a4>:10: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  model.backward(output, train_classes[random_batches[batch]])


RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [54]:
x = torch.ones(2,3)
print(x[1][1])

tensor(1.)
